## Cohort Extraction Notebook
### Christopher V. Cosgriff, MPH
#### NYU School of Medicine
<hr />
In this notebook we will extract the necessary variables for feature engineering and cohort construction to build a set of severity score models for this study.


The initial variables we will extract are as follows:
* Age, gender, weight, ethnicity (`patient` table)
* Source of admission, unit type (`patient` table)
* Laboratory data on the first day  (`labsfirstday` materialized view)
    * Blood gases: PaO2, pH, base excess, bicarbonate,
    * Hematology: hematocrit, hemoglobin, lymphocytes, neutrophils, platelets, white cell count
    * Electrolytes: calcium, chloride, ionized calcium, magnesium, phosphate, sodium
    * Biochemistry: albumin, amylase, bilirubin, blood urea nitrogen (BUN), B-natriuretic peptide, creatine phosphokinase (cpk), creatinine, lactate, lipase, troponin I/T, pH, bicarbonate, base excess, glucose
    * Coagulation: PT/INR, fibrinogen
* Vital signs on the first day (`vitalsfirstday` materialized view)
    * Heart rate
    * Blood pressure
    * Respiratory rate
    * SpO2
* Treatments (`treatmentfirstday` materializd view)
    * Antiarrhythmics
    * Antibiotics
    * Vasopressors
    * Sedatives
    * Diuretics
    * Blood products
* Ventilation status (`apachepredvar` table)
* Admission Dx (`APACHE_GROUPS` materialized view)
* APACHE IVa Features (`apachepredvar` table)

The _label_ for our classifier as well as their APACHE IVa predicted mortality are located in `apachepatientresult`.

## 0 - Environment

In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sklearn.model_selection import train_test_split

dbname = 'eicu'
schema_name = 'eicu_crd'
query_schema = 'SET search_path TO ' + schema_name + ';'
con = psycopg2.connect(dbname=dbname)

## 1 - Materialized Views

We will generate the requisite materialized views to aid in the extraction of the cohort features. We start by introducing helper functions for interacting with the eICU-CRD.

In [2]:
def execute_query_safely(sql, con):
    cur = con.cursor()
    try:
        cur.execute(sql)
    except:
        cur.execute('rollback;')
        raise
    finally:
        cur.close()
    return

def generate_materialized_view(query_file, con, query_schema):
    with open(query_file) as fp:
        query = ''.join(fp.readlines())
    print('Generating materialized view using {} ...'.format(query_file), end=' ')
    execute_query_safely(query_schema + query, con)
    print('done.')

Now we generate the materialized views.

In [3]:
generate_materialized_view('./sql/vitalsfirstday.sql', con, query_schema)
generate_materialized_view('./sql/labsfirstday.sql', con, query_schema)
generate_materialized_view('./sql/treatmentfirstday.sql', con, query_schema)
generate_materialized_view('./sql/apache-groups.sql', con, query_schema)

Generating materialized view using ./sql/vitalsfirstday.sql ... done.
Generating materialized view using ./sql/labsfirstday.sql ... done.
Generating materialized view using ./sql/treatmentfirstday.sql ... done.
Generating materialized view using ./sql/apache-groups.sql ... done.


## 2 - Load Features

We begin by loading a base cohort. As our most import exclusion criteria is whether or not they had an APACHE IVa score (so we can fit models in the subpopulations), we can join the `patient` table on the `apachepredvar` and `apachepatientresult` table. There are 200,859 ICU stays, and so the number of rows returned will be the remainder after exclusion of patients for which APACHE data or an APACHE score is not available.

In [4]:
base_cohort_query = query_schema + '''
WITH apacheIV AS
(
    SELECT patientunitstayid, apachescore
         , CAST(predictedhospitalmortality AS float) AS apache_prediction
         , actualhospitalmortality 
    FROM apachepatientresult
    WHERE apacheversion = 'IVa'
    AND apachescore > 0
    AND CAST(predictedhospitalmortality AS float) >= 0
)
, admit_order AS
(
    SELECT patientunitstayid, uniquepid
    , ROW_NUMBER() OVER (partition BY uniquepid ORDER BY hospitaladmitoffset DESC, patientunitstayid) AS admission_num
    FROM patient
)

SELECT p.patientunitstayid, p.age, p.gender, p.ethnicity, p.admissionheight AS height
       , p.admissionweight AS weight , p.unittype AS unit_type, p.unitadmitsource
       , p.unitdischargeoffset AS unit_los, p.hospitaldischargeoffset AS hospital_los
       , a.day1meds AS gcs_meds, a.day1verbal AS gcs_verbal, a.day1motor AS gcs_motor
       , a.day1eyes AS gcs_eyes, a.admitDiagnosis AS admit_diagnosis, ag.apachedxgroup as adx_group
       , a.thrombolytics AS apache_thrombolytics, a.electivesurgery AS apache_elect_surg
       , a.readmit AS apache_readmit, ao.admission_num, a.ima AS apache_ima
       , a.midur AS apache_midur, a.oOBVentDay1 AS apache_ventday1, a.oOBIntubDay1 AS apache_intubday1
       , a.day1fio2 AS apache_fio2, a.day1pao2 AS apache_pao2, (a.day1pao2 / a.day1fio2) AS apache_o2ratio
       , a.ejectfx AS apache_ejectfx, a.creatinine AS apache_creatinine
       , a.graftcount AS apache_graftcount, o.apache_prediction
       , o.apachescore AS apache_score, o.actualhospitalmortality AS hospital_expiration
FROM patient p
INNER JOIN apachepredvar a
ON p.patientunitstayid = a.patientunitstayid
INNER JOIN apacheIV o
ON p.patientunitstayid = o.patientunitstayid
INNER JOIN admit_order ao
on p.patientunitstayid = ao.patientunitstayid
INNER JOIN APACHE_GROUPS ag
on p.patientunitstayid = ag.patientunitstayid
ORDER BY patientunitstayid;
'''

base_cohort = pd.read_sql_query(base_cohort_query, con)
base_cohort.shape

(136231, 33)

Of the 200,859 patients in the database, 136,231 have APACHE IVa variables recorded as well as a hospital mortality prediction carried out. We'll then load the variables that will be used to derive the expanded feature set into a dataframe. 

In [5]:
feature_set_query = query_schema + '''
SELECT v.patientunitstayid, v.HR_Mean, v.SBP_periodic_Mean, v.DBP_periodic_Mean
    , v.MAP_periodic_Mean, v.SBP_aperiodic_Mean, v.DBP_aperiodic_Mean
    , v.MAP_aperiodic_Mean, v.RR_Mean, v.SpO2_Mean, v.TempC_Mean
    , ANIONGAP_min, ANIONGAP_max, ALBUMIN_min, ALBUMIN_max 
    , AMYLASE_min, AMYLASE_max, BASEEXCESS_min, BASEEXCESS_max
    , BICARBONATE_min, BICARBONATE_max, BUN_min, BUN_max, BNP_min
    , BNP_max, CPK_min, CPK_max, BILIRUBIN_min, BILIRUBIN_max
    , CALCIUM_min, CALCIUM_max, IONCALCIUM_min, IONCALCIUM_max
    , CREATININE_min, CREATININE_max, CHLORIDE_min, CHLORIDE_max
    , GLUCOSE_min, GLUCOSE_max, HEMATOCRIT_min, HEMATOCRIT_max
    , FIBRINOGEN_min, FIBRINOGEN_max, LIPASE_min, LIPASE_max
    , HEMOGLOBIN_min, HEMOGLOBIN_max, LACTATE_min, LACTATE_max
    , LYMPHS_min, LYMPHS_max, MAGNESIUM_min, MAGNESIUM_max
    , PAO2_min, PAO2_max, PH_min, PH_max, PLATELET_min
    , PLATELET_max, PMN_min, PMN_max, PHOSPHATE_min, PHOSPHATE_max
    , POTASSIUM_min, POTASSIUM_max, PTT_min, PTT_max, INR_min
    , INR_max, PT_min, PT_max, SODIUM_min, SODIUM_max
    , TROPI_min, TROPI_max, TROPT_min, TROPT_max, WBC_min
    , WBC_max, t.abx, t.pressor, t.antiarr, t.sedative
    , t.diuretic, t.blood_product
FROM vitalsfirstday v
LEFT JOIN labsfirstday l
ON v.patientunitstayid = l.patientunitstayid
LEFT OUTER JOIN treatmentfirstday t
ON v.patientunitstayid = t.patientunitstayid;
'''

feature_set = pd.read_sql_query(feature_set_query, con)
feature_set.shape

(192320, 85)

We can then merge the two dataframes. Note that we are using an _inner join_ here and so if a patient did not have a vital sign recordin in `vitalsperiodic` they will not be included in the cohort.

In [6]:
cohort = pd.merge(left=base_cohort, right=feature_set, how='inner', on='patientunitstayid')
cohort.shape

(134946, 117)

Here we go from 136,231 to 134,946. The missing 1,285 did not have recorded vitals and their APACHE IVa score was likely derived from the `nursecharting` table which we are not utilizing in this study.

## 3 - Inclusion / Exclusion

By nature of our SQL query, we have already excluded patients not eligible/capable of producing a valid score, and patients who lack all vital/lab/treatment data. We can then check the APACHE IVa criteria:

1. Not readmissions
2. Not admitted from another ICU
3. Admitted to ICU for $\geq4$hours
4. Not burn patients
5. Not transplant patients (other than hepatic renal)
5. Age $\geq16$

__1 - Not Readmitted__

In [7]:
cohort = cohort.loc[cohort.apache_readmit == 0, :]
cohort.shape

(134946, 117)

There are no readmissions.

__2 - Not Admitted from ICU__

In [8]:
cohort = cohort.loc[cohort.unitadmitsource != 'Other ICU', :]
cohort.shape

(134890, 117)

56 patients were admitted from another ICU, but still had scores calculated.

__3 - ICU LoS $\geq$ 4h__

In [9]:
cohort = cohort.loc[cohort.unit_los >= 240, :]
cohort.shape

(134890, 117)

No patients had a LoS <4h.

__4 - Not Burn Patients__ 

In [10]:
cohort = cohort.loc[cohort.admit_diagnosis != 'BURN', :]
cohort.shape

(134890, 117)

There were no burn patients inappropriately kept in the cohort.

__5 - Not Transplant Patients__ 

In [11]:
cohort = cohort.loc[cohort.admit_diagnosis != 'KIDPANTRAN', :]
cohort = cohort.loc[cohort.admit_diagnosis != 'S-KIDPTRAN', :]
cohort = cohort.loc[cohort.admit_diagnosis != 'S-HEARTRAN', :]
cohort = cohort.loc[cohort.admit_diagnosis != 'HEARTRAN', :]
cohort = cohort.loc[cohort.admit_diagnosis != 'S-LUNGTRAN', :]
cohort = cohort.loc[cohort.admit_diagnosis != 'LUNGTRAN', :]
cohort = cohort.loc[cohort.admit_diagnosis != 'LUNGSTRAN', :]
cohort.shape

(134890, 117)

There were also no transplant patients inappropriately kept in the cohort.

__6 - Age $\geq$ 16, and Not Missing__

In [12]:
# Per Rodrigo, the median age for >89 pt in eICU is 93
cohort.loc[cohort.age == '> 89', 'age'] = 93.0
cohort = cohort.loc[cohort.age != '', :]
cohort.age = cohort.age.astype('float64')
cohort = cohort.loc[cohort.age >= 16., :]
cohort.shape

(134890, 117)

No patients <16 years of age were in the dataset to be excluded.

## 4 - Cleaning, Formatting and Feature Engineering

Before anything else, we can drop numerous variables from the pull as we won't need both min/max for various features (they'll often be the same value since many things are only measured once in 24 hours, and even if they're not they'll be highly correlated). Instead we'll be going by the following principle: the most _abnormal_ laboratory value in the first 24 hours of ICU stay will be included. That is we will use:

* The minimum value for: bicarbonate, chloride, calcium, magnesium, base excess (including negative values), platelets, hemoglobin, phosphate, fibrinogen, pH and hematocrit
* The maximum value for: creatinine, BUN, bilirubin, PT/INR, lactate, troponin I/T, amylase, lipase, B-natriuretic peptide and creatinine phosphokinase
* For sodium, which aberrantly deviates bidirectionally, the most abnormal value was defined as the value with greatest deviation from the normal range boundaries.
    * This can be applied to glucose and potassium as well.
* For white blood cell and neutrophil counts, if any measurements were lower than the lower limit of normal, the minimum value was used; if the minimum was within normal range then the maximum was used as the most abnormal value.

In [13]:
# for the unidirectional abberations, just drop what isn't needed
lab_drop = ['bicarbonate_max', 'chloride_max', 'calcium_max', 'magnesium_max', 
            'baseexcess_max', 'platelet_max', 'hemoglobin_max', 'phosphate_max', 
            'fibrinogen_max', 'ph_max', 'hematocrit_max', 'creatinine_min',
            'bun_min', 'bilirubin_min', 'pt_min', 'inr_min', 'lactate_min', 
            'tropi_min', 'tropt_min', 'amylase_min', 'lipase_min', 'bnp_min',
            'cpk_min', 'albumin_max', 'ioncalcium_max', 'pao2_max', 'pt_min',
            'ptt_min', 'inr_min', 'aniongap_min']
cohort = cohort.drop(lab_drop, axis=1)

# sodium, deviates bidirectionally
sodium_check = abs(cohort.sodium_min - 135.) >= abs(cohort.sodium_max - 145.)
sodium = np.empty(len(cohort.index), dtype='float64')
sodium[sodium_check] = cohort.sodium_min[sodium_check]
sodium[~sodium_check] = cohort.sodium_max[~sodium_check]
cohort = cohort.assign(sodium=sodium)
cohort = cohort.drop(['sodium_min', 'sodium_max'], axis=1)

# potassium, deviates bidirectionally, same treatment then
potassium_check = abs(cohort.potassium_min - 3.5) >= abs(cohort.potassium_max - 5.0)
potassium = np.empty(len(cohort.index), dtype='float64')
potassium[potassium_check] = cohort.potassium_min[potassium_check]
potassium[~potassium_check] = cohort.potassium_max[~potassium_check]
cohort = cohort.assign(potassium=potassium)
cohort = cohort.drop(['potassium_min', 'potassium_max'], axis=1)

# similar treatment for glucose since hyperglycemia and hypoglycemia can both
# be important dependent on the clinical context.
glucose_check = abs(cohort.glucose_min - 70) >= abs(cohort.glucose_max - 130)
glucose = np.empty(len(cohort.index), dtype='float64')
glucose[glucose_check] = cohort.glucose_min[glucose_check]
glucose[~glucose_check] = cohort.glucose_max[~glucose_check]
cohort = cohort.assign(glucose=glucose)
cohort = cohort.drop(['glucose_min', 'glucose_max'], axis=1)

# wbc counts
wbc_check = cohort.wbc_min < 2
pmn_check = cohort.pmn_min < 45
lym_check = cohort.lymphs_min < 20

wbc = np.empty(len(cohort.index), dtype='float64')
wbc[wbc_check] = cohort.wbc_min[wbc_check]
wbc[~wbc_check] = cohort.wbc_max[~wbc_check]
cohort = cohort.assign(wbc=wbc)
cohort = cohort.drop(['wbc_min', 'wbc_max'], axis=1)

pmn = np.empty(len(cohort.index), dtype='float64')
pmn[pmn_check] = cohort.pmn_min[pmn_check]
pmn[~pmn_check] = cohort.pmn_max[~pmn_check]
cohort = cohort.assign(pmn=pmn)
cohort = cohort.drop(['pmn_min', 'pmn_max'], axis=1)

lym = np.empty(len(cohort.index), dtype='float64')
lym[lym_check] = cohort.lymphs_min[lym_check]
lym[~lym_check] = cohort.lymphs_max[~lym_check]
cohort = cohort.assign(lym=lym)
cohort = cohort.drop(['lymphs_min', 'lymphs_max'], axis=1)

Inspection of the data reveals some variables which are redundant and others which appear unreliable, and in some cases one source is better than another. As such we will:
* Use aperiodic BP instead of periodic
* Drop temperature; it is an unreliable signal when automatically captured
* Drop APACHE elective surgery, we will simply classify patients by their admission diagnosis
* Drop PaO2/FiO2 from APACHE and instead just use the PaO2 values derived directly from laboratory data
* Use sCr from labs instead of APACHE table
* Drop LoS variables since they would let our models peek into the future
* Drop unit admit source, it was only included for the exclusion criteria
* Drop APACHE score
* Drop APACHE readmit and admission number, as they were only used to examine exclusion criteria
* Missingness will be handled during modeling, but we should make sure that all missingness is labeled with `np.nan` and not -1 as is present in some of the eICU tables

In [14]:
cohort = cohort.replace(-1, np.nan)
cohort = cohort.drop(['sbp_periodic_mean', 'dbp_periodic_mean', 'map_periodic_mean',
                      'tempc_mean', 'apache_elect_surg', 'apache_creatinine', 'apache_pao2', 
                      'apache_fio2', 'apache_o2ratio', 'hospital_los', 'unit_los', 
                      'unitadmitsource', 'apache_score', 'apache_readmit', 'admission_num'], axis=1)

We next turn to formatting the data so that it will be amenable to modeling. This entails converting categorical variables into indicators, and thus we must first convert the strings composing the categories into good variable names.

We'll start with admission diagnoses.

In [15]:
cohort = cohort.drop('admit_diagnosis', axis=1)
adx_dummies = pd.get_dummies(cohort.adx_group, 'adx', drop_first=True)
cohort = pd.concat([cohort, adx_dummies], axis=1)
cohort = cohort.drop('adx_group', axis=1)

Next we turn to gender and ethnicity.

In [16]:
male_gender = (cohort.gender == 'Male').astype('int')
cohort = cohort.assign(male_gender=male_gender)
cohort = cohort.drop('gender', axis=1)

eth_map = {'Caucasian' : 'caucasian', 'Other/Unknown' : 'other', 
           'Native American' : 'native_american', 'African American' : 'african_american',
          'Asian' : 'asian', 'Hispanic' : 'hispanic', '' : 'other'}
cohort.ethnicity = cohort.ethnicity.map(eth_map)
eth_dummies = pd.get_dummies(cohort.ethnicity, 'eth', drop_first=True)
cohort = pd.concat([cohort, eth_dummies], axis=1)
cohort = cohort.drop('ethnicity', axis=1)

This leaves unit type as a categorical variable.

In [17]:
cohort.unit_type = cohort.unit_type.str.replace('-', '_')
cohort.unit_type = cohort.unit_type.str.replace(' ', '_')
unit_dummies = pd.get_dummies(cohort.unit_type, 'unit', drop_first=True)
cohort = pd.concat([cohort, unit_dummies], axis=1)
cohort = cohort.drop('unit_type', axis=1)

## 5 - Save Train/Test Split of Features and Label

We need to save the label and remove it from the features. We'll also need to save the APACHE prediction so as to incorporate the APACHE IVa model in our analysis.

In [18]:
label = (cohort.hospital_expiration == 'EXPIRED').astype('int')
apache_pred = cohort.apache_prediction
cohort = cohort.drop(['hospital_expiration', 'apache_prediction'], axis=1)

And now we can form a train test split.

In [19]:
train_X, test_X, train_y, test_y, train_apache, test_apache = train_test_split(cohort, label, apache_pred, test_size=0.25, random_state=42)

With that, we can save the CSV files corresponding to data frames we generated.

In [20]:
train_X.to_csv('./data/train_X.csv', index=False)
train_y.to_csv('./data/train_y.csv', index=False, header=True)
train_apache.to_csv('./data/train_apache.csv', index=False, header=True)

test_X.to_csv('./data/test_X.csv', index=False)
test_y.to_csv('./data/test_y.csv', index=False, header=True)
test_apache.to_csv('./data/test_apache.csv', index=False, header=True)

With this portion complete, we can move onto the construction of our mortality models.